In [ ]:
import tensorflow as tf
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#taking file zip dataset in google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
local_zip = '/content/gdrive/MyDrive/Training_Images_ODIR5K.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/train_dataset')
zip_ref.close()

local_zip = '/content/gdrive/MyDrive/Testing_Images_ODIR5K.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/test_dataset')
zip_ref.close()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/content/tmp/train_dataset"
training_datagen = ImageDataGenerator(rescale = 1./255)

TESTING_DIR = "/content/tmp/test_dataset"
testing_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical',
  batch_size=126
)

test_generator = testing_datagen.flow_from_directory(
    TESTING_DIR,
    target_size=(150,150),
    class_mode='categorical',
  batch_size=126
)

In [ ]:
model.fit(
    train_generator,
    epochs = 10,
    validation_data = test_generator
)